In [1]:
import requests
from bs4 import BeautifulSoup
from os.path  import basename
import time as t
from random import randrange as rr
import json
import numpy as np
import pandas as pd

C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [3]:
# season_list = [x for x in range(2010, 2019)]
# season_list

In [4]:
# season_list = [x for x in range(2010, 2019)]

# for season in season_list:
#     page = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

#     # beginn der Zeitmessung und übergabe des html dokuments
#     start = t.time()
#     tree = requests.get(page, headers = headers)
#     soup = BeautifulSoup(tree.content, 'html.parser')
    
#     club_list = []


#     for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#         try:
#             club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#         except: KeyError

#     club_list = list(set(club_list))
#     print(club_list)
#     t.sleep(3)

In [5]:
# club_list = []


# for i in range(len(soup.select('a.vereinprofil_tooltip'))):
#     try:
#         club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
#     except: KeyError
        
# club_list = list(set(club_list))
# print(club_list)

In [6]:
# page = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/2662/saison_id/2018/plus/1'


# # beginn der Zeitmessung und übergabe des html dokuments
# start = t.time()
# tree = requests.get(page, headers = headers)
# soup = BeautifulSoup(tree.content, 'html.parser')


In [7]:
game_list = []
season_list = [x for x in range(2009, 2019)]


for season in season_list:
    print(season)
    t.sleep(1)
    page = 'https://www.transfermarkt.at/bundesliga/startseite/wettbewerb/A1/plus/?saison_id=' + str(season)

    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    
    club_list = []


    for i in range(len(soup.select('a.vereinprofil_tooltip'))):
        try:
            club_list.append(soup.select('a.vereinprofil_tooltip')[i].attrs['id'])
        except: KeyError

    club_list = list(set(club_list))
        

    for club in club_list:
        t.sleep(5)
        page = 'https://www.transfermarkt.at/red-bull-salzburg/spielplan/verein/' + str(club) + '/saison_id/' + str(season) + '/plus/1'
        # beginn der Zeitmessung und übergabe des html dokuments
        start = t.time()
        tree = requests.get(page, headers = headers)
        soup = BeautifulSoup(tree.content, 'html.parser')


        for i in range(3, len(soup.select('div.box'))):
            box = soup.select('div.box')[i]
            competition = soup.select('div.box')[i]('div')[0]('img')[0].attrs['alt']
            saison = str(season) + '/' + str(int(str(season)[-2:])+1)

            for j in range(1, len(box.select('tr'))):
                line = box.select('tr')[j]

                gameday = line.select('td')[0].get_text().strip()
                date = line.select('td')[1].get_text()
                time = line.select('td')[2].get_text()
                home_team_id = line.select('td')[3]('a')[0].attrs['id']
                home_team_name = line.select('td')[3]('img')[0].attrs['alt']
                guest_team_id = line.select('td')[5]('a')[0].attrs['id']
                guest_team_name = line.select('td')[5]('img')[0].attrs['alt']
                game_system = line.select('td')[7].get_text()
                if club == home_team_id:
                    home_coach = line.select('td')[8].get_text()
                    guest_coach = ''
                else:
                    home_coach = ''
                    guest_coach = line.select('td')[8].get_text()
                try:
                    viewers = int(line.select('td')[9].get_text().replace('.', ''))
                except:
                    viewers = 0
                result = line.select('td')[10].get_text().strip()


                game_list.append({
                    'saison': saison,
                    'competition': competition,
                    'gameday': gameday,
                    'date': date,
                    'time': time,
                    'home_team_id': home_team_id,
                    'home_team_name': home_team_name,
                    'guest_team_id': guest_team_id,
                    'guest_team_name': guest_team_name,
                    'game_system': game_system,
                    'home_coach': home_coach,
                    'guest_coach': guest_coach,
                    'viewers': viewers,
                    'result': result
                })

                #print(game_list[-1])

2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [15]:
game_list[0], len(game_list)

({'saison': '2009/10',
  'competition': 'Europa League',
  'gameday': 'Gruppe C',
  'date': 'Do. 17.09.2009',
  'time': '19:00',
  'home_team_id': '170',
  'home_team_name': 'SK Rapid Wien',
  'guest_team_id': '41',
  'guest_team_name': 'Hamburger SV',
  'game_system': '4-5-1',
  'home_coach': 'Peter Pacult',
  'guest_coach': '',
  'viewers': 49850,
  'result': '3:0'},
 4385)

In [16]:
df = pd.DataFrame(game_list)

In [17]:
df.head()

,competition,date,game_system,gameday,guest_coach,guest_team_id,guest_team_name,home_coach,home_team_id,home_team_name,result,saison,time,viewers
0,Europa League,Do. 17.09.2009,4-5-1,Gruppe C,,41,Hamburger SV,Peter Pacult,170,SK Rapid Wien,3:0,2009/10,19:00,49850
1,Europa League,Do. 01.10.2009,4-5-1,Gruppe C,Peter Pacult,170,SK Rapid Wien,,371,Celtic Glasgow,1:1,2009/10,21:05,42000
2,Europa League,Do. 22.10.2009,4-5-1,Gruppe C,Peter Pacult,170,SK Rapid Wien,,1017,Hapoel Tel Aviv,5:1,2009/10,21:05,13000
3,Europa League,Do. 05.11.2009,4-2-3-1,Gruppe C,,1017,Hapoel Tel Aviv,Peter Pacult,170,SK Rapid Wien,0:3,2009/10,19:00,47000
4,Europa League,Mi. 02.12.2009,4-5-1,Gruppe C,Peter Pacult,170,SK Rapid Wien,,41,Hamburger SV,2:0,2009/10,21:05,46000


In [11]:
df.tail()

,competition,date,game_system,gameday,guest_coach,guest_team_id,guest_team_name,home_coach,home_team_id,home_team_name,result,saison,time,viewers
4380,Bundesliga,So. 26.05.2019,3-4-3 flach,32,Robert Ibertsberger,14,FK Austria Wien,,413,LASK,5:2,2018/19,17:00,6087
4381,ÖFB-Cup,So. 22.07.2018,4-3-1-2,1.Runde,Thomas Letsch,14,FK Austria Wien,,12083,FV Austria XIII,0:4,2018/19,11:00,2152
4382,ÖFB-Cup,Mi. 26.09.2018,4-2-3-1,2.Runde,,122,SK Sturm Graz,Thomas Letsch,14,FK Austria Wien,2:0,2018/19,20:45,8900
4383,ÖFB-Cup,Di. 30.10.2018,4-1-3-2,Achtelfinale,,7443,Floridsdorfer AC,Thomas Letsch,14,FK Austria Wien,3:1,2018/19,20:20,7212
4384,ÖFB-Cup,Fr. 15.02.2019,4-4-2 doppel 6,Viertelfinale,Thomas Letsch,14,FK Austria Wien,,316,Grazer AK 1902,2:1,2018/19,19:00,12295


In [12]:
df[df.result.str.len() > 3]

,competition,date,game_system,gameday,guest_coach,guest_team_id,guest_team_name,home_coach,home_team_id,home_team_name,result,saison,time,viewers
9,Europa League Qualifikation,Do. 06.08.2009,4-3-3 defensiv,3.Runde Rückspiele,Peter Pacult,170,SK Rapid Wien,,8554,APOP Kinyras Pegias,2:2 n.V.,2009/10,19:15,2500
48,ÖFB-Cup,Sa. 15.08.2009,4-4-2,1.Runde,Peter Pacult,170,SK Rapid Wien,,4320,SC/ESV Parndorf,2:3 n.V.,2009/10,17:00,4500
128,ÖFB-Cup,Mi. 31.03.2010,4-4-2 doppel 6,Viertelfinale,Helmut Kraft,413,LASK,,17444,SC Wiener Neustadt,2:1 n.V.,2009/10,19:00,1450
168,ÖFB-Cup,Mi. 31.03.2010,5-4-1,Viertelfinale,,541,SC Austria Lustenau,Paul Gludovatz,266,SV Ried,5:3 n.E.,2009/10,19:00,1666
218,ÖFB-Cup,Sa. 15.08.2009,4-5-1,1.Runde,Franco Foda,122,SK Sturm Graz,,17556,TSV St. Johann,2:4 n.V.,2009/10,18:30,3000
221,ÖFB-Cup,Di. 30.03.2010,4-4-2 Raute,Viertelfinale,,503,FC Admira Wacker Mödling,Franco Foda,122,SK Sturm Graz,1:0 n.V.,2009/10,19:00,6700
302,ÖFB-Cup,Fr. 18.09.2009,?,2.Runde,Helmut Kraft,17444,SC Wiener Neustadt,,935,FC Pasching,4:6 n.E.,2009/10,19:00,550
304,ÖFB-Cup,Mi. 31.03.2010,4-4-2 Raute,Viertelfinale,,413,LASK,Peter Schöttel,17444,SC Wiener Neustadt,2:1 n.V.,2009/10,19:00,1450
406,Europa League Qualifikation,Do. 27.08.2009,4-4-2,4.Runde Rückspiele,,2783,Metalurh Donetsk,Karl Daxbacher,14,FK Austria Wien,3:2 n.V.,2009/10,19:15,10450
495,ÖFB-Cup,So. 19.09.2010,4-4-2 doppel 6,2.Runde,Peter Pacult,170,SK Rapid Wien,,4383,FK Austria Wien Amateure,4:5 n.E.,2010/11,15:00,3200


In [13]:
df.to_csv('Ö-Bundesliga-2010-2018.csv')

In [14]:
with open('Ö-Bundesliga-2010-2018.json', 'w', encoding='utf-8') as f:
    json.dump(game_list, f, ensure_ascii=False, indent=4)